# SPIRE-NEP master catalogue: Flags

In [1]:
import numpy as np
from astropy.table import Table

import itertools

from herschelhelp_internal.flagging import flag_outliers

In [2]:
SUFFIX = "20180220"
FIELD = "SPIRE-NEP"
catname = "../../dmu1/dmu1_ml_SPIRE-NEP/data/master_catalogue_spire-nep_{}.fits".format(SUFFIX)
master_catalogue = Table.read(catname)

In [3]:
all_bands = []

## 1. Magnitudes and magnitude erros

In [4]:
def flag_mag(flagcol, mask):
    
    # Add flag columns if does not exist
    if flagcol not in master_catalogue.colnames:
        master_catalogue[flagcol] = np.zeros(len(master_catalogue), dtype=bool)
    
    # Flagged
    master_catalogue[flagcol][mask] = np.ones(len(mask), dtype=bool)
    print('    Number of flagged objects:', len(master_catalogue[flagcol][mask]))

### 1.a Pan-STARRS Aperture and Total magnitude errors

In [5]:
## dmu0: Pan-STARRS forced photometry catalogue
gpc1_err = 0.0010860000038519502
bands = ["GPC1 g", "GPC1 r", "GPC1 i", "GPC1 z", "GPC1 y"]

for i, band in enumerate(bands):
    print(band)
    basecol = band.replace(" ", "_").lower()
    
    ecol_ap, ecol_tot = "merr_ap_{}".format(basecol), "merr_{}".format(basecol)
    flagcol_ap, flagcol_tot = "flag_ap_{}".format(basecol), "flag_{}".format(basecol)
    
    mask_ap  = np.where(master_catalogue[ecol_ap]  == gpc1_err)[0]
    mask_tot = np.where(master_catalogue[ecol_tot] == gpc1_err)[0]
    
    print('  Aperture magnitude')
    flag_mag(flagcol_ap, mask_ap)
    print('  Total magnitude')
    flag_mag(flagcol_tot, mask_tot)
    

GPC1 g
  Aperture magnitude
    Number of flagged objects: 6
  Total magnitude
    Number of flagged objects: 7
GPC1 r
  Aperture magnitude
    Number of flagged objects: 3
  Total magnitude
    Number of flagged objects: 3
GPC1 i
  Aperture magnitude
    Number of flagged objects: 2
  Total magnitude
    Number of flagged objects: 2
GPC1 z
  Aperture magnitude
    Number of flagged objects: 4
  Total magnitude
    Number of flagged objects: 3
GPC1 y
  Aperture magnitude
    Number of flagged objects: 3
  Total magnitude
    Number of flagged objects: 3


## 3. Save table

In [ ]:
#Merge any aperture flags
for col in master_catalogue.colnames:
    if col.startswith("flag_ap_"):
        try:
            master_catalogue[col.replace("_ap_", "_")] = (master_catalogue[col.replace("_ap_", "_")] |
                                                          master_catalogue[col])
            master_catalogue.remove_column(col)
        except KeyError:
            print("{} only has aperture flags.".format(col))
            master_catalogue.rename_column(col, col.replace("_ap_", "_"))

In [6]:
flag_cols = ["help_id"]
for col in master_catalogue.colnames:
    if col.startswith("flag_"):
        flag_cols += [col]
new_catname = "./data/{}_{}_flags.fits".format(FIELD.lower(),SUFFIX)
master_catalogue[flag_cols].write(new_catname, overwrite = True)